In [1]:
import MGTomo.model as mgmodel
#import MGTomo.functions as fcts
import numpy as np
import astra
import MGTomo.tomoprojection as mgproj
from MGTomo.utils import myexp, mylog, mydiv

import torch

from MGTomo.functions import kl_distance as KL

ModuleNotFoundError: No module named 'MGTomo'

## Testing the autograd wrapper for matvec in TomoTorch (vererbt von TomoParallel)

In [2]:
model = mgmodel.astra_model(63,{'mode' : 'line', 'num_angles' : 50, 'level_decrease' : 1})
fine_dim = model.dim
A = model.proj_factory(fine_dim)

x_orig = np.random.randn(63, 63)

In [3]:
ATensor = mgproj.TomoTorch(A)
x_torch = torch.from_numpy(x_orig)
x_torch.requires_grad = True

output = ATensor(x_torch)
x_0 = torch.ones(63,63)
x_0.requires_grad = True

In [4]:
(output - torch.from_numpy(A.matvec(x_orig))).abs().max()

tensor(0., dtype=torch.float64, grad_fn=<MaxBackward1>)

In [5]:
torch.autograd.gradcheck(ATensor.forward, (x_torch, ),eps = 0.001, atol = 1e-3)

GradcheckError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[0.9999, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9995, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9999,  ..., 0.0000, 0.0000, 1.0023],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)
analytical:tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0020],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)
